# The Battle of the Neighborhoods
Problem

Traveling to a new city as exciting and vibrant as Paris can be overwhelming. Utilizing an itinerary can maximize valuable time, prioritize must-do activities, and plan for easier and faster travel between attractions. This project aims to create an efficient itinerary that saves time, includes must-see attractions, and make suggestions for what to do, see, and eat, taking into account distance, time, and location. 


Interest

More than 83 million visitors make the pilgrimage to Paris per year. These visitors would be interested in the results of this project for their next visit to Paris. Even Parisians can find benefits for exploring their hometown or a new arrondissement with the results of our analysis using location data.


In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import json
import xml
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins

import json
from pprint import pprint

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge

The following packages will be DOWNGRADED:

    folium: 0.9.1-py_0 conda-forge --> 0.5.0-py_0 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris#Arrondissements').text

In [3]:
soup = BeautifulSoup(url, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Arrondissements of Paris - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Arrondissements_of_Paris","wgTitle":"Arrondissements of Paris","wgCurRevisionId":901799808,"wgRevisionId":901799808,"wgArticleId":1175882,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 French-language sources (fr)","Articles with short description","Use dmy dates from March 2016","Arrondissements of Paris","Government of Paris"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Fe

In [11]:
arrond_table = soup.find("table", class_ = 'wikitable sortable')
arrond_table_rows = arrond_table.find_all('tr')

In [12]:
information = []
for row in arrond_table_rows:
    info = row.text.split('\n')[1:-1]
    information.append(info)

del information[0][1::2] # There are extra column headers "" that we don't need, so delete them    

information

[['Arrondissement (R for Right Bank, L for Left Bank)',
  'Name',
  'Area (km2)',
  'Population(March 1999 census)',
  'Population(July 2005 estimate)',
  'Density (2005)(inhabitants per km2)',
  'Peak of population',
  'Mayor'],
 ['1st (Ie) R',
  'Louvre',
  '1.826\xa0km2 (0.705\xa0sq\xa0mi)',
  '16,888',
  '17,700',
  '9,693',
  'before 1861',
  'Jean-François Legaret'],
 ['2nd (IIe) R',
  'Bourse',
  '0.992\xa0km2 (0.383\xa0sq\xa0mi)',
  '19,585',
  '20,700',
  '20,867',
  'before 1861',
  'Jacques Boutault'],
 ['3rd (IIIe) R',
  'Temple',
  '1.171\xa0km2 (0.452\xa0sq\xa0mi)',
  '34,248',
  '35,100',
  '29,974',
  'before 1861',
  'Pierre Aidenbaum'],
 ['4th (IVe) R',
  'Hôtel-de-Ville',
  '1.601\xa0km2 (0.618\xa0sq\xa0mi)',
  '30,675',
  '28,600',
  '17,864',
  'before 1861',
  'Christophe Girard'],
 ['5th (Ve) L',
  'Panthéon',
  '2.541\xa0km2 (0.981\xa0sq\xa0mi)',
  '58,849',
  '60,600',
  '23,849',
  '1911',
  'Florence Berthout'],
 ['6th (VIe) L',
  'Luxembourg',
  '2.154\xa0km

In [13]:
arrond_df = pd.DataFrame(information[1:], columns=information[0])
# where information[1:] contains each row of neighborhoods
# and columns = information[0] gives the column names

arrond_df.head(25)

,"Arrondissement (R for Right Bank, L for Left Bank)",Name,Area (km2),Population(March 1999 census),Population(July 2005 estimate),Density (2005)(inhabitants per km2),Peak of population,Mayor
0,1st (Ie) R,Louvre,1.826 km2 (0.705 sq mi),"16,888","17,700","9,693",before 1861,Jean-François Legaret
1,2nd (IIe) R,Bourse,0.992 km2 (0.383 sq mi),"19,585","20,700","20,867",before 1861,Jacques Boutault
2,3rd (IIIe) R,Temple,1.171 km2 (0.452 sq mi),"34,248","35,100","29,974",before 1861,Pierre Aidenbaum
3,4th (IVe) R,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"30,675","28,600","17,864",before 1861,Christophe Girard
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"58,849","60,600","23,849",1911,Florence Berthout
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"44,919","45,200","20,984",1911,Jean-Pierre Lecoq
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"56,985","55,400","13,552",1926,None
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"39,314","38,700","9,972",1891,None
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"55,838","58,500","26,847",1901,None
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"89,612","88,800","30,705",1881,None


In [14]:
# Add a column with just the Arrondissement numbers

arrond_number = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th','10th',
                '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th']

Arr = pd.DataFrame({'Arrondissement': arrond_number})

arrond_df = pd.concat([arrond_df, Arr], axis = 1)

arrond_df.head()

,"Arrondissement (R for Right Bank, L for Left Bank)",Name,Area (km2),Population(March 1999 census),Population(July 2005 estimate),Density (2005)(inhabitants per km2),Peak of population,Mayor,Arrondissement
0,1st (Ie) R,Louvre,1.826 km2 (0.705 sq mi),"16,888","17,700","9,693",before 1861,Jean-François Legaret,1st
1,2nd (IIe) R,Bourse,0.992 km2 (0.383 sq mi),"19,585","20,700","20,867",before 1861,Jacques Boutault,2nd
2,3rd (IIIe) R,Temple,1.171 km2 (0.452 sq mi),"34,248","35,100","29,974",before 1861,Pierre Aidenbaum,3rd
3,4th (IVe) R,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"30,675","28,600","17,864",before 1861,Christophe Girard,4th
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"58,849","60,600","23,849",1911,Florence Berthout,5th


In [15]:
arrond_df.dtypes

Arrondissement (R for Right Bank, L for Left Bank)    object
Name                                                  object
Area (km2)                                            object
Population(March 1999 census)                         object
Population(July 2005 estimate)                        object
Density (2005)(inhabitants per km2)                   object
Peak of population                                    object
Mayor                                                 object
Arrondissement                                        object
dtype: object

In [16]:
def landmark_loc(list_of_landmarks):
    for place in list_of_landmarks:
        address = (place)
        geolocator = Nominatim(user_agent="Paris_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{''}, {}, {},'.format(place,latitude,longitude))

landmark_loc(landmarks)

NameError: name 'landmarks' is not defined

In [ ]:
landmarks_loc = ['Cathédrale Notre-Dame de Paris', 48.85293695, 2.35005149954546,
'Arc de Triomphe', 48.8737791, 2.29503722603767,
'Centre Pompidou', 48.8605918, 2.35247431766534,
'Sacré-Cœur Basilica', 48.88680575, 2.34301534488351,
'Eiffel Tower', 48.8582602, 2.29449905431968,
'The Louvre', 48.8611473, 2.33802768704666,
'La Grande Arche', 48.8930151, 2.2376206,
'Les Invalides', 48.8570374, 2.3118779,
'Fondation Louis Vuitton', 48.87665005, 2.26333809580246,
'Panthéon', 48.8454189, 2.35258442491778,
'Picasso Museum', 41.3851066, 2.18120508674285,
'Cinémathèque Française', 48.8369987, 2.38263520273537,
'Opéra Garnier', 48.87203085, 2.3317901855896,
'The Grand Palais', 48.86616135, 2.31222295966943,
'Hôtel de Ville', 43.9492974, 4.80526012041732,
'Fondation Cartier', 48.8373182, 2.33193184219906,
'Jeu de Paume', 48.8658156, 2.32408776132074,
'Jardin des Plantes', 48.8432224, 2.35950895709484,
'Fondation Jerôme Seydoux-Pathé', 48.8334765, 2.35490681339522,
'Île de la Cité', 48.85470505, 2.3474846899114,
'The Palais de Tokyo', 48.8639905, 2.29652151313114,
'Philharmonie de Paris', 48.89192125, 2.39413864295307,
'Musée d’Orsay', 48.85991785, 2.32658283469656,
'Palais du Luxembourg', 48.84852845, 2.33683647732996,
'Montmartre', 46.8021347, -0.7198939,
'Champs-Élysées', 48.8707573, 2.3053312,
'Palace of Versailles', 48.80442525, 2.1202852709926,
'Tuileries Garden', 48.86366215, 2.32683861305756,
'Moulin Rouge', 48.8840787, 2.3324082,
'Disneyland Paris', 48.8711359, 2.77612651196751,
'Place de la Concorde', 29.9002052, -90.0095146,
'Catacombs of Paris', 48.833964, 2.3324924,
'Rodin Museum', 39.9619295, -75.1739508,
'Île de la Cité', 48.85470505, 2.3474846899114,
'Orangerie Museum', 48.8637655, 2.32266020780693,
'Pont des Arts', 48.8578114, 2.3372459,
'Île Saint-Louis', 48.85161305, 2.35673414953666,
'Place Vendôme', 48.8674634, 2.32942811682519,
'Saint-Jacques Tower', 48.8580032, 2.34892171290132,
'La Conciergerie', 48.8559492, 2.3460263]

landmarks_df = pd.DataFrame(np.array(landmarks_loc).reshape(40, 3), columns = ["Landmark","Latitude","Longitude"])

# Change the latitude and longitude to a float
landmarks_df['Latitude'] = landmarks_df.Latitude.astype(float)
landmarks_df['Longitude'] = landmarks_df.Longitude.astype(float)

In [17]:
# turn numbered arrondissements into a list
arrond_list = arrond_df['Arrondissement'].unique()
arrond_list

def landmark_loc(list_of_arronds):
    for place in list_of_arronds:
        address = (place + ' Arrondissement')
        geolocator = Nominatim(user_agent="arrond_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('{}, {}, {},'.format(place,latitude,longitude))

landmark_loc(arrond_list)

1st, 48.8646144, 2.334396,
2nd, 48.868743, 2.341688,
3rd, 48.864212, 2.360936,
4th, 48.8562021, 2.3556193,
5th, 48.8460591, 2.3445228,
6th, 48.8504333, 2.3329507,
7th, 48.8570281, 2.3201953,
8th, 48.8773158, 2.3172611,
9th, 48.876019, 2.339962,
10th, 48.876106, 2.35991,
11th, 48.858416, 2.379703,
12th, 48.8396154, 2.3957517,
13th, 48.8323973, 2.3555829,
14th, 48.8330589, 2.3269591,
15th, 48.8413705, 2.3003827,
16th, 48.8631709, 2.2757648,
17th, 48.8842239, 2.3223639,
18th, 48.8900117, 2.3464668,
19th, 48.889343, 2.38436,
20th, 48.865042, 2.398929,


In [18]:
# turn numbered arrondissements into a list
arrond_list = arrond_df['Arrondissement'].unique()
arrond_list

def landmark_loc(list_of_arronds):
    LL = []
    for place in list_of_arronds:
        address = (place + ' Arrondissement')
        geolocator = Nominatim(user_agent="arrond_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        LL.append(place)
        LL.append(latitude)
        LL.append(longitude)
    print(LL)


LL = landmark_loc(arrond_list)

['1st', 48.8646144, 2.334396, '2nd', 48.868743, 2.341688, '3rd', 48.864212, 2.360936, '4th', 48.8562021, 2.3556193, '5th', 48.8460591, 2.3445228, '6th', 48.8504333, 2.3329507, '7th', 48.8570281, 2.3201953, '8th', 48.8773158, 2.3172611, '9th', 48.876019, 2.339962, '10th', 48.876106, 2.35991, '11th', 48.858416, 2.379703, '12th', 48.8396154, 2.3957517, '13th', 48.8323973, 2.3555829, '14th', 48.8330589, 2.3269591, '15th', 48.8413705, 2.3003827, '16th', 48.8631709, 2.2757648, '17th', 48.8842239, 2.3223639, '18th', 48.8900117, 2.3464668, '19th', 48.889343, 2.38436, '20th', 48.865042, 2.398929]


In [19]:
# not sure why the LL above couldn't be reshaped, so copied them below to insert into a dataframe converter

LL = ['1st', 48.8646144, 2.334396, '2nd', 48.868743, 2.341688, '3rd', 48.864212, 2.360936, 
      '4th', 48.8562021, 2.3556193, '5th', 48.8460591, 2.3445228, '6th', 48.8504333, 2.3329507, 
      '7th', 48.8570281, 2.3201953, '8th', 48.8773158, 2.3172611, '9th', 48.876019, 2.339962, 
      '10th', 48.876106, 2.35991, '11th', 48.858416, 2.379703, '12th', 48.8396154, 2.3957517, 
      '13th', 48.8323973, 2.3555829, '14th', 48.8330589, 2.3269591, '15th', 48.8413705, 2.3003827, 
      '16th', 48.8631709, 2.2757648, '17th', 48.8842239, 2.3223639, '18th', 48.8900117, 2.3464668, 
      '19th', 48.889343, 2.38436, '20th', 48.865042, 2.398929]

In [20]:
arrond_LL = pd.DataFrame(np.array(LL).reshape(20,3), columns = ["Arrondissement","Latitude","Longitude"])

# Change the latitude and longitude to a float
arrond_LL['Latitude'] = arrond_LL.Latitude.astype(float)
arrond_LL['Longitude'] = arrond_LL.Longitude.astype(float)

arrond_LL

,Arrondissement,Latitude,Longitude
0,1st,48.864614,2.334396
1,2nd,48.868743,2.341688
2,3rd,48.864212,2.360936
3,4th,48.856202,2.355619
4,5th,48.846059,2.344523
5,6th,48.850433,2.332951
6,7th,48.857028,2.320195
7,8th,48.877316,2.317261
8,9th,48.876019,2.339962
9,10th,48.876106,2.359910


In [21]:
with open('arrondissements_osm_paris.geojson') as file:    
    paris = json.load(file)
    
paris

FileNotFoundError: [Errno 2] No such file or directory: 'arrondissements_osm_paris.geojson'

In [9]:
address = ('Paris, France')
geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{''}, {}, {},'.format(address,latitude,longitude))

Paris, France, 48.8566101, 2.3514992,


In [5]:
paris_lat = 48.8566101
paris_long = 2.3514992

paris_map = folium.Map(location=[paris_lat, paris_long], zoom_start = 12)
paris_map

In [ ]:
# create a map of Paris
map_landmarks2 = folium.Map(location = [paris_lat, paris_long], zoom_start = 10)

map_landmarks2.choropleth(
    geo_data=paris,
    data=pop_2005,
    columns=['Arrondissement', 'Population'],
    key_on='feature.properties.nom',
    fill_color='PuBu', 
    fill_opacity=0.1, 
    line_opacity=0.5,
)

#add landmarks markers to the Paris map
for lat, long, landmark in zip(landmarks_df['Latitude'], landmarks_df['Longitude'], 
                                 landmarks_df['Landmark']):
    label = '{}'.format(landmark)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_landmarks2)
    
#add neighborhood markers to the Paris map
for lat, long, arrond in zip(arrond_LL['Latitude'], arrond_LL['Longitude'], 
                                 arrond_LL['Arrondissement']):
    label = '{}'.format(arrond)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 5, 
        popup = label,
        color = 'white',
        fill = True,
        fill_color = 'green',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_landmarks2)
        
map_landmarks2

In [27]:
CLIENT_ID = 'Z0ZXZVGI032SZ5BMIHQOLXDRNQ0FUABOLMGVYMQAZ1ZP4W34' # your Foursquare ID
CLIENT_SECRET = '10EGZBZJMQJJBQCSJ1PR40PUUOJXKKVF2RTYHGLAPY1M4L40' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [28]:
# Make dataframe of the most popular arrondissement


popular_arrond = arrond_LL[arrond_LL['Arrondissement'].isin(
    ['1st', '3rd', '4th', '5th', '6th', '7th', '8th', '18th'])]

popular_arrond

,Arrondissement,Latitude,Longitude
0,1st,48.864614,2.334396
2,3rd,48.864212,2.360936
3,4th,48.856202,2.355619
4,5th,48.846059,2.344523
5,6th,48.850433,2.332951
6,7th,48.857028,2.320195
7,8th,48.877316,2.317261
17,18th,48.890012,2.346467


In [29]:
# Make dataframe of the most popular arrondissement


popular_arrond = arrond_LL[arrond_LL['Arrondissement'].isin(
    ['1st', '3rd', '4th', '5th', '6th', '7th', '8th', '18th'])]

popular_arrond

,Arrondissement,Latitude,Longitude
0,1st,48.864614,2.334396
2,3rd,48.864212,2.360936
3,4th,48.856202,2.355619
4,5th,48.846059,2.344523
5,6th,48.850433,2.332951
6,7th,48.857028,2.320195
7,8th,48.877316,2.317261
17,18th,48.890012,2.346467


In [30]:
# Example call of 1st arrondissement

arrond_1_lat = arrond_LL.iloc[0,1]

arrond_1_long = arrond_LL.iloc[0,2]

LIMIT = 250

radius = 2000



url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    arrond_1_lat, 
    arrond_1_long, 
    VERSION, 
    radius, 
    LIMIT)

url1

'https://api.foursquare.com/v2/venues/explore?client_id=Z0ZXZVGI032SZ5BMIHQOLXDRNQ0FUABOLMGVYMQAZ1ZP4W34&client_secret=10EGZBZJMQJJBQCSJ1PR40PUUOJXKKVF2RTYHGLAPY1M4L40&ll=48.8646144,2.334396&v=20180605&radius=2000&limit=250'

In [31]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5d0dc0d207190200251bad6b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f0b706de5e8207020057ed7-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/ramen_',
          'suffix': '.png'},
         'id': '55a59bace4b013909087cb2a',
         'name': 'Udon Restaurant',
         'pluralName': 'Udon Restaurants',
         'primary': True,
         'shortName': 'Udon'}],
       'id': '4f0b706de5e8207020057ed7',
       'location': {'address': "9 rue d'Argenteuil",
        'cc': 'FR',
        'city': 'Paris',
        'country': 'France',
        'distance': 44,
        'formattedAddress': ["9 rue d'Argenteuil", '75001 Paris', 'France'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.86471330060748,
          'lng': 2.

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
            

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Arrondissement', 
                  'Arrondissement Latitude', 
                  'Arrondissement Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
popular_arrond_venues = getNearbyVenues(names=popular_arrond['Arrondissement'],
                                   latitudes=popular_arrond['Latitude'],
                                   longitudes=popular_arrond['Longitude'],
                                   radius=2000, LIMIT=300
                                  )

1st
3rd
4th
5th
6th
7th
8th
18th


In [34]:
arrond_venues = getNearbyVenues(names=arrond_LL['Arrondissement'],
                                   latitudes=arrond_LL['Latitude'],
                                   longitudes=arrond_LL['Longitude'],
                                   radius=2000, LIMIT=300
                                  )

1st
2nd
3rd
4th
5th
6th
7th
8th
9th
10th
11th
12th
13th
14th
15th
16th
17th
18th
19th
20th


In [35]:
print(arrond_venues.shape)
arrond_venues

(2000, 7)


,Arrondissement,Arrondissement Latitude,Arrondissement Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1st,48.864614,2.334396,Sanukiya,48.864713,2.333805,Udon Restaurant
1,1st,48.864614,2.334396,Brasserie Réjane,48.865486,2.334824,Restaurant
2,1st,48.864614,2.334396,Jardin du Palais Royal,48.864941,2.337728,Garden
3,1st,48.864614,2.334396,Comédie-Française,48.863088,2.336612,Theater
4,1st,48.864614,2.334396,Ellsworth,48.865528,2.337057,French Restaurant
5,1st,48.864614,2.334396,Le Roch Hotel & Spa Paris,48.866200,2.332995,Hotel
6,1st,48.864614,2.334396,Les Arts Décoratifs,48.863077,2.333393,Art Museum
7,1st,48.864614,2.334396,Palais Royal,48.863758,2.337121,Historic Site
8,1st,48.864614,2.334396,Nolinski,48.865367,2.334584,Hotel
9,1st,48.864614,2.334396,JanTchi,48.866001,2.335977,Korean Restaurant


In [36]:
# one hot encoding
arrond_onehot = pd.get_dummies(arrond_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
arrond_onehot['Arrondissement'] = arrond_venues['Arrondissement'] 

# move neighborhood column to the first column
fixed_columns = [arrond_onehot.columns[-1]] + list(arrond_onehot.columns[:-1])
arrond_onehot = arrond_onehot[fixed_columns]

arrond_onehot.head()

,Arrondissement,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Caribbean Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Record Shop,Recording Studio,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,1st,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1st,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1st,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
arrond_grouped = arrond_onehot.groupby('Arrondissement').mean().reset_index()
arrond_grouped

,Arrondissement,African Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Caribbean Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,Gourmet Shop,Greek Restaurant,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangxi Restaurant,Juice Bar,Korean Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Record Shop,Recording Studio,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,10th,0.00,0.00,0.02,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.06,0.03,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.05,0.02,0.01,0.00,0.00,0.02,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.01,0.03,0.00,0.00,0.00,0.04,0.03,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.02,0.00,0.04,0.01,0.01,0.00
1,11th,0.00,0.00,0.00,0.03,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.03,0.04,0.00,0.00,0.00,0.01,0

In [38]:
num_top_venues = 10

for hood in arrond_grouped['Arrondissement']:
    print("----"+hood+"----")
    temp = arrond_grouped[arrond_grouped['Arrondissement'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----10th----
                           venue  freq
0                          Hotel  0.07
1              French Restaurant  0.07
2                         Bakery  0.06
3                   Cocktail Bar  0.05
4             Italian Restaurant  0.04
5                       Wine Bar  0.04
6  Vegetarian / Vegan Restaurant  0.04
7                 Breakfast Spot  0.03
8            Indie Movie Theater  0.03
9            Japanese Restaurant  0.03


----11th----
                venue  freq
0   French Restaurant  0.15
1            Wine Bar  0.07
2        Cocktail Bar  0.06
3         Pizza Place  0.05
4  Italian Restaurant  0.04
5                 Bar  0.04
6         Art Gallery  0.03
7              Bistro  0.03
8              Bakery  0.03
9         Coffee Shop  0.03


----12th----
                  venue  freq
0     French Restaurant  0.11
1              Wine Bar  0.05
2                 Hotel  0.04
3                   Bar  0.04
4    Italian Restaurant  0.03
5                Bakery  0.03
6         

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Arrondissement']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
arrond_venues_sorted = pd.DataFrame(columns=columns)
arrond_venues_sorted['Arrondissement'] = arrond_grouped['Arrondissement']

for ind in np.arange(arrond_grouped.shape[0]):
    arrond_venues_sorted.iloc[ind, 1:] = return_most_common_venues(arrond_grouped.iloc[ind, :], num_top_venues)

arrond_venues_sorted

,Arrondissement,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10th,French Restaurant,Hotel,Bakery,Cocktail Bar,Wine Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Indie Movie Theater,Japanese Restaurant,Breakfast Spot
1,11th,French Restaurant,Wine Bar,Cocktail Bar,Pizza Place,Italian Restaurant,Bar,Coffee Shop,Bakery,Art Gallery,Bistro
2,12th,French Restaurant,Wine Bar,Bar,Hotel,Gym / Fitness Center,Italian Restaurant,Bookstore,Farmers Market,Bakery,Park
3,13th,French Restaurant,Vietnamese Restaurant,Hotel,Thai Restaurant,Chinese Restaurant,Italian Restaurant,Plaza,Café,Bar,Bakery
4,14th,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Creperie,Bistro,Ice Cream Shop,Auvergne Restaurant,Park,Pizza Place
5,15th,French Restaurant,Hotel,Bakery,Japanese Restaurant,Korean Restaurant,Italian Restaurant,Dessert Shop,Plaza,Bistro,Gym / Fitness Center
6,16th,French Restaurant,Hotel,Italian Restaurant,Bakery,Plaza,Art Museum,Japanese Restaurant,Pizza Place,Café,Burger Joint
7,17th,French Restaurant,Italian Restaurant,Hotel,Wine Bar,Bakery,Bar,Plaza,Bistro,Beer Bar,Coffee Shop
8,18th,French Restaurant,Italian Restaurant,Bakery,Plaza,Bar,Coffee Shop,Bistro,Pizza Place,Theater,Beer Bar
9,19th,French Restaurant,Multiplex,Bar,Concert Hall,Music Venue,Cocktail Bar,Park,Pizza Place,Café,Restaurant


In [41]:
# run K-means to cluster the arrondissements into 5 clusters

# set number of clusters
kclusters = 5

arrond_grouped_clustering = arrond_LL.drop('Arrondissement', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(arrond_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 3, 3, 3, 0, 0, 2], dtype=int32)

In [42]:
# add clustering labels
#arrond_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

arrond_merge = arrond_LL

# merge grouped with data to add latitude/longitude for each arrondissement
arrond_merge = arrond_merge.join(arrond_venues_sorted.set_index('Arrondissement'), on='Arrondissement')

#arrond_merge = arrond_merge.drop('Cluster',1) # accidentally added added another cluster column so this was to get rid of that extra column

arrond_merge.head(20) # check the last columns!

,Arrondissement,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1st,48.864614,2.334396,Hotel,French Restaurant,Plaza,Art Museum,Cocktail Bar,Historic Site,Boutique,Theater,Pastry Shop,Pedestrian Plaza
1,2nd,48.868743,2.341688,Hotel,French Restaurant,Plaza,Cocktail Bar,Art Museum,Historic Site,Theater,Pedestrian Plaza,Women's Store,Bar
2,3rd,48.864212,2.360936,French Restaurant,Bookstore,Italian Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Breakfast Spot,Hotel,Sandwich Place,Wine Bar
3,4th,48.856202,2.355619,French Restaurant,Cocktail Bar,Plaza,Ice Cream Shop,Coffee Shop,Japanese Restaurant,Bookstore,Sandwich Place,Art Gallery,Wine Bar
4,5th,48.846059,2.344523,French Restaurant,Plaza,Ice Cream Shop,Wine Bar,Restaurant,Cocktail Bar,Coffee Shop,Bistro,Tea Room,Seafood Restaurant
5,6th,48.850433,2.332951,French Restaurant,Plaza,Hotel,Art Museum,Ice Cream Shop,Cocktail Bar,Garden,Wine Bar,Coffee Shop,Bookstore
6,7th,48.857028,2.320195,French Restaurant,Hotel,Plaza,Art Museum,Garden,Historic Site,Pastry Shop,Boutique,Italian Restaurant,Cocktail Bar
7,8th,48.877316,2.317261,Hotel,French Restaurant,Boutique,Gourmet Shop,Historic Site,Plaza,Bar,Japanese Restaurant,Theater,Tea Room
8,9th,48.876019,2.339962,French Restaurant,Hotel,Italian Restaurant,Cocktail Bar,Plaza,Indie Movie Theater,Theater,Bistro,Bakery,Pizza Place
9,10th,48.876106,2.359910,French Restaurant,Hotel,Bakery,Cocktail Bar,Wine Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Indie Movie Theater,Japanese Restaurant,Breakfast Spot


In [ ]:
cluster1 = arrond_merge.loc[arrond_merge['Cluster Label'] == 0]

cluster1

In [ ]:
cluster2 = arrond_merge.loc[arrond_merge['Cluster Label'] == 1]

cluster2

In [ ]:
cluster3 = arrond_merge.loc[arrond_merge['Cluster Label'] == 2]

cluster3

In [ ]:
cluster4 = arrond_merge.loc[arrond_merge['Cluster Label'] == 3]

cluster4

In [ ]:

cluster5 = arrond_merge.loc[arrond_merge['Cluster Label'] == 4]

cluster5

In [ ]:
## run K-means to cluster the landmarks into 5 clusters

# set number of clusters
kclusters = 8

landmark_clustering = landmarks_df.drop('Landmark', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(landmark_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

In [ ]:
# add clustering labels

#landmarks_df.insert(0, 'Cluster Label', kmeans.labels_)

landmarks_merge = landmarks_df

landmarks_merge = landmarks_merge.drop('Cluster',1) # accidentally added added another cluster column so this was to get rid of that extra column

landmarks_merge = landmarks_merge.sort_values(['Cluster Label'], ascending=False)

landmarks_merge

In [ ]:
arrond_venues

In [ ]:
# Things to Do
museum = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Museum','Art Museum', 'Science Museum', 'Exhibit', 'Art Gallery', 'History Museum'])]

outdoor_sights = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Bridge', 'Street Art','Plaza', 'Pedestrian Plaza', 'Scenic Lookout', 'Outdoor Sculpture', 'Theme Park Ride / Attraction', 'Canal', 'Park', 'Garden', 'Botanical Garden' ])]
                                   
fashion_store = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Department Store','Boutique',"Men's Store","Women's Store",'Cosmetics Shop', 'Clothing Store', 'Jewelry Store', 'Tailor Shop', 'Gift Shop', 'Shopping Mall'])]

night_life = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Nightclub', 'Hookah Bar', 'Jazz Club', 'Rock Club'])]

bike = arrond_venues.loc[arrond_venues['Venue Category'] == 'Bike Rental / Bike Share']                                   

relax = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Spa', 'Massage Studio'])]


# Things to Eat and Drink
asian_food = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Udon Restaurant', 'Japanese Restaurant', 'Asian Restaurant', 
                                                                     'Szechuan Restaurant', 'Korean Restaurant', 'Vietnamese Restaurant',
                                                                    'Thai Restaurant', 'Cambodian Restaurant', 'Sushi Restaurant',
                                                                    'Jiangxi Restaurant', 'Chinese Restaurant', 'Indian Restaurant'])]

world_food = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Modern European Restaurant', 'Italian Restaurant', 'Argentinian Restaurant', 'American Restaurant', 'Scandinavian Restaurant', 'Trattoria/Osteria' , 
                                                                     'Israeli Restaurant', 'Vegetarian / Vegan Restaurant', 'Seafood Restaurant', 'Lebanese Restaurant', 'Ethiopian Restaurant', 'Mexican Restaurant',
                                                                    'Breton Restaurant','Corsican Restaurant', 'Mediterranean Restaurant', 'Russian Restaurant', 'Tapas Restaurant', 'Falafel Restaurant', 'Moroccan Restaurant',
                                                                    'Turkish Restaurant', 'Portuguese Restaurant', 'Greek Restaurant', 'Brazilian Restaurant', 'Peruvian Restaurant', 'Comfort Food Restaurant', 'Steakhouse',
                                                                    'Basque Restaurant', 'Middle Eastern Restaurant', 'Arepa Restaurant', 'Southern / Soul Food Restaurant', 'African Restaurant', 'Gluten-free Restaurant'])]
                               
french = arrond_venues.loc[arrond_venues['Venue Category'].isin(['French Restaurant', 'Creperie','Cheese Shop', 'Provençal Restaurant', 'Bistro', 'Brasserie', 'Auvergne Restaurant'])]

dessert = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Ice Cream Shop','Pastry Shop', 'Bakery', 'Dessert Shop', 'Donut Shop', 'Chocolate Shop', 'Cupcake Shop'])] 
                           
drink = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Cocktail Bar', 'Beer Bar','Beer Store', 'Roof Deck', 'Wine Bar', 'Tea Room', 'Liquor Store', 'Wine Shop', 
                                                                'Bar', 'Gastropub', 'Speakeasy', 'Lounge', 'Beer Garden', 'Irish Pub', 'Champagne Bar', 'Hotel Bar', 'Vineyard'])]
cafe = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Café', 'Breakfast Spot', 'Coffe Shop', 'Tea Room'])]
                          
fast_food = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Burger Joint', 'Hot Dog Joint', 'Sandwich Place', 'Pizza Place', 
                                                                    'Fish & Chips Shop', 'Deli / Bodega', 'Taco Place', 'Food & Drink Shop',
                                                                   'BBQ Joint', 'Bagel Shop', 'Fast Food Restaurant', 'Diner', 'Food Truck'])]                 

market = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Farmers Market','Cheese Shop', 'Organic Grocery', 'Market', 'Supermarket'])]


# Things to See
historic_site = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Historic Site', 'Monument / Landmark'])]


# Things to hear
perform = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Theater', 'Indie Movie Theater', 'Opera House', 'Comedy Club', 'Concert Hall', 'Music Venue', 'Performing Arts Venue'])]                 


# Places to live 
lodging = arrond_venues.loc[arrond_venues['Venue Category'].isin(['Hotel', 'Hostel'])]

In [ ]:
# create a map of Paris

def map_with_arrond_and_points(points_df, arrondll):

    map_name = folium.Map(location = [48.8566101, 2.3514992], zoom_start = 12)
    
    map_name.choropleth(
        geo_data=paris,
        data=pop_2005,
        columns=['Arrondissement', 'Population'],
        key_on='feature.properties.nom',
        fill_color='PuBu', 
        fill_opacity=0.1, 
        line_opacity=0.5,)

#add points of interest markers to the Paris map
    for lat, long, points, cat in zip(points_df['Venue Latitude'], points_df['Venue Longitude'], 
                                 points_df['Venue'], points_df['Venue Category']):
        label = '{}, {}'.format(points, cat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, long],
            radius = 7, 
            popup = label,
            color = 'yellow',
            fill = True,
            fill_color = 'blue',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_name)
        
#add arrond markers to the Paris map
    for lat, long, arrond in zip(arrondll['Latitude'], arrondll['Longitude'], 
                                 arrondll['Arrondissement']):
        label = '{}'.format(arrond)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, long],
            radius = 5, 
            popup = label,
            color = 'white',
            fill = True,
            fill_color = 'green',
            fill_opacity = 0.5,
            parse_html = False).add_to(map_name)
        
        
    return map_name

In [ ]:
map_with_arrond_and_points(points_df = french, arrondll = arrond_LL)

In [ ]:
map_with_arrond_and_points(points_df = cafe, arrondll = arrond_LL)

In [ ]:
def map_with_arrond_and_points_v1(points_df, arrondll):

# Add the boundaries of the arrondissement
    map_name = folium.Map(location = [48.8566101, 2.3514992], zoom_start = 12)
    
    map_name.choropleth(
        geo_data=paris,
        data=pop_2005,
        columns=['Arrondissement', 'Population'],
        key_on='feature.properties.nom',
        fill_color='PuBu', 
        fill_opacity=0.1, 
        line_opacity=0.5,)

    # instantiate a mark cluster object for the incidents in the dataframe
    venues = plugins.MarkerCluster().add_to(map_name)

    # loop through the dataframe and add each data point to the mark cluster
    for lat, lng, name, cat in zip(points_df['Venue Latitude'], points_df['Venue Longitude'], 
                                 points_df['Venue'], points_df['Venue Category']):
        label = '{}, {}'.format(name, cat)
        label = folium.Popup(label, parse_html=True)
        folium.Marker(
            location=[lat, lng],
            icon=None,
            popup=label).add_to(venues)
        
        
#add arrond markers to the Paris map
    for lat, long, arrond in zip(arrondll['Latitude'], arrondll['Longitude'], 
                                 arrondll['Arrondissement']):
        label = '{}'.format(arrond)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, long],
            radius = 5, 
            popup = label,
            color = 'white',
            fill = True,
            fill_color = 'green',
            fill_opacity = 0.5,
            parse_html = False).add_to(map_name)
        
    return map_name

In [ ]:
def map_with_arrond_and_points_v2(points_df, arrondll, landmarks):

# Add the boundaries of the arrondissement
    map_name = folium.Map(location = [48.8566101, 2.3514992], zoom_start = 12)
    
    map_name.choropleth(
        geo_data=paris,
        data=pop_2005,
        columns=['Arrondissement', 'Population'],
        key_on='feature.properties.nom',
        fill_color='PuBu', 
        fill_opacity=0.1, 
        line_opacity=0.5,)

    # instantiate a mark cluster object for the incidents in the dataframe
    venues = plugins.MarkerCluster().add_to(map_name)

    # loop through the dataframe and add each data point to the mark cluster
    for lat, lng, name, cat in zip(points_df['Venue Latitude'], points_df['Venue Longitude'], 
                                 points_df['Venue'], points_df['Venue Category']):
        label = '{}, {}'.format(name, cat)
        label = folium.Popup(label, parse_html=True)
        folium.Marker(
            location=[lat, lng],
            icon=None,
            popup=label).add_to(venues)
        
        
#add arrond markers to the Paris map
    for lat, long, arrond in zip(arrondll['Latitude'], arrondll['Longitude'], 
                                 arrondll['Arrondissement']):
        label = '{}'.format(arrond)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, long],
            radius = 5, 
            popup = label,
            color = 'white',
            fill = True,
            fill_color = 'green',
            fill_opacity = 0.5,
            parse_html = False).add_to(map_name)
        
#add landmarks of interest markers to the Paris map
    for lat, long, landmark in zip(landmarks['Latitude'], landmarks['Longitude'], 
                                 landmarks['Landmark']):
        label = '{}'.format(landmark)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, long],
            radius = 7, 
            popup = label,
            color = 'red',
            fill = True,
            fill_color = 'blue',
            fill_opacity = 0.7,
            parse_html = False).add_to(map_name)
        
    return map_name

In [ ]:
map_with_arrond_and_points_v2(points_df = french, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = museum, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = market, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = dessert, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = lodging, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v1(points_df = historic_site, arrondll = arrond_LL)

In [ ]:
map_with_arrond_and_points_v1(points_df = outdoor_sights, arrondll = arrond_LL)

In [ ]:
map_with_arrond_and_points_v2(points_df = night_life, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = drink, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = cafe, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = fast_food, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points(drink, arrond_LL)

In [ ]:
map_with_arrond_and_points(fashion_store, arrond_LL)

In [ ]:
map_with_arrond_and_points_v2(points_df = world_food, arrondll = arrond_LL, landmarks = landmarks_df)

In [ ]:
map_with_arrond_and_points_v2(points_df = perform, arrondll = arrond_LL, landmarks = landmarks_df)